In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  # Needed for plotting

# def align_sequences(X, y):
#     if X.shape[0] > y.shape[0]:
#         X = X[:y.shape[0], :]
#     elif y.shape[0] > X.shape[0]:
#         delta = y.shape[0] - X.shape[0]
#         cut_start = delta // 2
#         cut_end = delta - cut_start
#         y = y[cut_start: -cut_end if cut_end > 0 else None, :]
#     return X, y

# def load_trial_data(imu_trial_folder, angles_file, window_size):
#     imu_dfs = []

#     # Load each IMU CSV
#     for fname in sorted(os.listdir(imu_trial_folder)):
#         if fname.endswith(".csv"):
#             path = os.path.join(imu_trial_folder, fname)
#             df = pd.read_csv(path)

#             print(f"\n--- Raw IMU File: {fname} ---")
#             print(df)

#             # Drop timestamp (first column)
#             df = df.drop(columns=[df.columns[0]])
#             imu_dfs.append(df)

#     if not imu_dfs:
#         raise RuntimeError(f"No IMU files found in {imu_trial_folder}")

#     # Determine the minimum shared length
#     lengths = [df.shape[0] for df in imu_dfs]
#     min_len = min(lengths)

#     print(f"[INFO] Truncating all IMU files to minimum length: {min_len} (original lengths: {lengths})")

#     # Truncate all IMU dataframes
#     imu_dfs = [df.iloc[:min_len, :] for df in imu_dfs]
#     imu_concat_df = pd.concat(imu_dfs, axis=1)

#     print(f"\n=== Concatenated IMU Data (Trial: {os.path.basename(imu_trial_folder)}) ===")
#     print(imu_concat_df)
#     print(f"Shape: {imu_concat_df.shape}")

#     X = imu_concat_df.values

#     # Load and clean angles
#     y_df = pd.read_csv(angles_file)
#     print(f"\n--- Angle File: {os.path.basename(angles_file)} ---")
#     print(y_df)

#     y = y_df.drop(columns=[col for col in y_df.columns if 'time' in col.lower()]).values

#     # Align sequence lengths between X and y
#     X, y = align_sequences(X, y)

#     # Optional: quick plot to verify y shape visually
#     plt.plot(y[:500])
#     plt.title("First 500 target values")
#     plt.show()

#     # Windowing
#     X_seq, y_seq = [], []
#     for i in range(len(X) - window_size + 1):
#         X_seq.append(X[i:i + window_size])
#         y_seq.append(y[i + window_size - 1])

#     return np.array(X_seq), np.array(y_seq)

In [2]:
import importlib.util

torch_spec = importlib.util.find_spec("torch")
print(torch_spec)

ModuleSpec(name='torch', loader=<_frozen_importlib_external.SourceFileLoader object at 0x705f1830c1c0>, origin='/home/fadluw/anaconda3/envs/misk/lib/python3.10/site-packages/torch/__init__.py', submodule_search_locations=['/home/fadluw/anaconda3/envs/misk/lib/python3.10/site-packages/torch'])


In [3]:
import sys
import os
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset

# Ensure 'scripts' is in the module path
sys.path.append('scripts')

# Import your preprocessor
from load_all_trials import load_and_preprocess_all_trials

# Define data sources
imu_data_root = os.path.abspath("dataset/imu_data")
angles_dir    = os.path.abspath("dataset/angles_data/csv_files")
window_size   = 50

# Load, align, scale, window all data across trials
X_all, y_all, scaler_X, scaler_y = load_and_preprocess_all_trials(
    imu_data_root, angles_dir, window_size
)

# Train/val split
# First split: train vs temp (val + test)
X_train, X_temp, y_train, y_temp = train_test_split(
    X_all, y_all, test_size=0.4, random_state=42
)

# Second split: val vs test (from temp)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

# Wrap in PyTorch datasets
train_dataset = TensorDataset(
    torch.tensor(X_train, dtype=torch.float32),
    torch.tensor(y_train, dtype=torch.float32)
)
val_dataset = TensorDataset(
    torch.tensor(X_val, dtype=torch.float32),
    torch.tensor(y_val, dtype=torch.float32)
)

# Sanity check
x0, y0 = train_dataset[0]
print("Sample input shape:", x0.shape)
print("Sample target shape:", y0.shape)


[INFO] Processing trial_1
[INFO] Processing trial_2
[INFO] Processing trial_3
[INFO] Processing trial_4
[INFO] Finished processing. Total sequences: 35174
Sample input shape: torch.Size([50, 12])
Sample target shape: torch.Size([24])


In [4]:
import torch.nn as nn

class IMULSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x: (batch_size, seq_len, input_size)
        out, _ = self.lstm(x)
        out = out[:, -1, :]  # Use last time step
        return self.fc(out)


In [5]:
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
import torch.nn as nn
import sys
import os
from datetime import datetime

# --- Optional: R² helper function ---
def r2_score(pred, target):
    target_mean = torch.mean(target, dim=0)
    ss_total = torch.sum((target - target_mean) ** 2)
    ss_res = torch.sum((target - pred) ** 2)
    return 1 - (ss_res / ss_total)


# --- Logging Setup ---
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
log_dir = os.path.join('logs', f'run_{timestamp}')
os.makedirs(log_dir, exist_ok=True)

LOG_PATH = os.path.join(log_dir, 'training_log.txt')
SAVE_STDOUT = True  # Toggle to also show logs in terminal

import csv

csv_path = os.path.join(log_dir, 'metrics.csv')
csv_fields = ['epoch', 'train_mse', 'train_mae', 'train_rmse', 'train_r2',
              'val_mse', 'val_mae', 'val_rmse', 'val_r2']

csv_file = open(csv_path, mode='w', newline='')
csv_writer = csv.DictWriter(csv_file, fieldnames=csv_fields)
csv_writer.writeheader()


from IPython.display import display  # Jupyter-safe
from io import StringIO

class DualLogger:
    def __init__(self, filepath, print_to_stdout=True):
        self.log_file = open(filepath, 'w')
        self.print_to_stdout = print_to_stdout

    def log(self, text):
        self.log_file.write(text + '\n')
        self.log_file.flush()
        if self.print_to_stdout:
            print(text)

    def close(self):
        self.log_file.close()

logger = DualLogger(LOG_PATH, print_to_stdout=True)



# Hyperparameters
hidden_size = 128
num_layers = 2
batch_size = 32
epochs = 20
learning_rate = 1e-3

# Header info
print("=" * 70)
print(f"Training started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")
print(f"Hyperparameters:")
print(f"  Hidden Size   = {hidden_size}")
print(f"  Num Layers    = {num_layers}")
print(f"  Batch Size    = {batch_size}")
print(f"  Epochs        = {epochs}")
print(f"  Learning Rate = {learning_rate}")
print(f"  Window Size   = {window_size}")
print("=" * 70)


# Save hyperparameters to a JSON file alongside the log
import json

hyperparams = {
    "hidden_size": hidden_size,
    "num_layers": num_layers,
    "batch_size": batch_size,
    "epochs": epochs,
    "learning_rate": learning_rate,
    "window_size": window_size,
}

with open(os.path.join(log_dir, 'hyperparameters.json'), 'w') as f:
    json.dump(hyperparams, f, indent=2)


# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Model and optimizer
input_size = train_dataset[0][0].shape[1]
output_size = train_dataset[0][1].shape[0]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = IMULSTMModel(input_size, hidden_size, num_layers, output_size).to(device)
optimizer = Adam(model.parameters(), lr=learning_rate)
mse_loss_fn = nn.MSELoss()
mae_loss_fn = nn.L1Loss()

# Training loop
logger.log(f"\nStarting training for {epochs} epochs...\n")
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    logger.log(f"Epoch {epoch+1}/{epochs}")
    train_loss_mse = 0
    train_loss_mae = 0
    train_r2_total = 0
    train_samples = 0

    for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        pred = model(X_batch)
        loss_mse = mse_loss_fn(pred, y_batch)
        loss_mae = mae_loss_fn(pred, y_batch)

        optimizer.zero_grad()
        loss_mse.backward()
        optimizer.step()

        batch_size_actual = X_batch.size(0)
        train_loss_mse += loss_mse.item() * batch_size_actual
        train_loss_mae += loss_mae.item() * batch_size_actual
        train_r2_total += r2_score(pred, y_batch).item() * batch_size_actual
        train_samples += batch_size_actual

        running_loss += loss_mse.item()
        if (batch_idx + 1) % 100 == 0 or (batch_idx + 1) == len(train_loader):
            avg_batch_loss = running_loss / min(100, (batch_idx + 1))
            logger.log(f"  Batch {batch_idx+1:>4}/{len(train_loader)} - Running MSE Loss: {avg_batch_loss:.4f}")
            running_loss = 0.0

    avg_train_mse = train_loss_mse / train_samples
    avg_train_mae = train_loss_mae / train_samples
    avg_train_rmse = avg_train_mse ** 0.5
    avg_train_r2 = train_r2_total / train_samples

    # Validation
    model.eval()
    val_loss_mse = 0
    val_loss_mae = 0
    val_r2_total = 0
    val_samples = 0

    with torch.no_grad():
        for X_val, y_val in val_loader:
            X_val, y_val = X_val.to(device), y_val.to(device)
            pred = model(X_val)

            loss_mse = mse_loss_fn(pred, y_val)
            loss_mae = mae_loss_fn(pred, y_val)

            val_loss_mse += loss_mse.item() * X_val.size(0)
            val_loss_mae += loss_mae.item() * X_val.size(0)
            val_r2_total += r2_score(pred, y_val).item() * X_val.size(0)
            val_samples += X_val.size(0)

    avg_val_mse = val_loss_mse / val_samples
    avg_val_mae = val_loss_mae / val_samples
    avg_val_rmse = avg_val_mse ** 0.5
    avg_val_r2 = val_r2_total / val_samples

    # Final epoch summary
    logger.log(f"  ↳ Train → MSE: {avg_train_mse:.4f}, MAE: {avg_train_mae:.4f}, RMSE: {avg_train_rmse:.4f}, R²: {avg_train_r2:.4f}")
    logger.log(f"  ↳ Val   → MSE: {avg_val_mse:.4f}, MAE: {avg_val_mae:.4f}, RMSE: {avg_val_rmse:.4f}, R²: {avg_val_r2:.4f}")
    logger.log("-" * 70)


    csv_writer.writerow({
        'epoch': epoch + 1,
        'train_mse': avg_train_mse,
        'train_mae': avg_train_mae,
        'train_rmse': avg_train_rmse,
        'train_r2': avg_train_r2,
        'val_mse': avg_val_mse,
        'val_mae': avg_val_mae,
        'val_rmse': avg_val_rmse,
        'val_r2': avg_val_r2,
    })
    csv_file.flush()  # Important to avoid loss on crashes

csv_file.close()




Training started: 2025-04-07 05:40:05
Device: CUDA
Hyperparameters:
  Hidden Size   = 128
  Num Layers    = 2
  Batch Size    = 32
  Epochs        = 20
  Learning Rate = 0.001
  Window Size   = 50

Starting training for 20 epochs...

Epoch 1/20
  Batch  100/660 - Running MSE Loss: 0.9593
  Batch  200/660 - Running MSE Loss: 0.8621
  Batch  300/660 - Running MSE Loss: 0.8372
  Batch  400/660 - Running MSE Loss: 0.7936
  Batch  500/660 - Running MSE Loss: 0.7553
  Batch  600/660 - Running MSE Loss: 0.6919
  Batch  660/660 - Running MSE Loss: 0.4210
  ↳ Train → MSE: 0.8064, MAE: 0.6120, RMSE: 0.8980, R²: 0.1786
  ↳ Val   → MSE: 0.6997, MAE: 0.5688, RMSE: 0.8365, R²: 0.2702
----------------------------------------------------------------------
Epoch 2/20
  Batch  100/660 - Running MSE Loss: 0.6818
  Batch  200/660 - Running MSE Loss: 0.6773
  Batch  300/660 - Running MSE Loss: 0.6474
  Batch  400/660 - Running MSE Loss: 0.6091
  Batch  500/660 - Running MSE Loss: 0.6276
  Batch  600/660 - 

In [6]:
train_y = train_dataset[:][1]
val_y = val_dataset[:][1]

print("Train y mean/std:", train_y.mean().item(), train_y.std().item())
print("Val   y mean/std:", val_y.mean().item(), val_y.std().item())
print("Train y min/max:", train_y.min().item(), train_y.max().item())
print("Val   y min/max:", val_y.min().item(), val_y.max().item())

Train y mean/std: -0.00016199485980905592 1.006033182144165
Val   y mean/std: 0.0032754098065197468 0.9947154521942139
Train y min/max: -5.084049701690674 10.121524810791016
Val   y min/max: -5.09799861907959 9.574485778808594


In [7]:
torch.save(model.state_dict(), "imu_lstm_weights.pth")


In [8]:
from torch.utils.data import DataLoader
import torch.nn as nn

# Define loss function if not already done
criterion = nn.MSELoss()

# Wrap test data (if not already done)
test_dataset = TensorDataset(
    torch.tensor(X_test, dtype=torch.float32),
    torch.tensor(y_test, dtype=torch.float32)
)

# Create test DataLoader
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Run evaluation
model.eval()
test_loss = 0
all_preds = []
all_targets = []

with torch.no_grad():
    for X_test_batch, y_test_batch in test_loader:
        X_test_batch = X_test_batch.to(device)
        y_test_batch = y_test_batch.to(device)

        pred = model(X_test_batch)
        loss = criterion(pred, y_test_batch)

        test_loss += loss.item() * X_test_batch.size(0)

        all_preds.append(pred.cpu())
        all_targets.append(y_test_batch.cpu())

# Final loss value (mean over all samples)
test_loss /= len(test_dataset)
print(f"Test MSE Loss: {test_loss:.4f}")

# Optional: Convert to NumPy for metrics or plotting
all_preds = torch.cat(all_preds).numpy()
all_targets = torch.cat(all_targets).numpy()



Test MSE Loss: 0.0668
